## Importing all the required libraries

In [156]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier,XGBRFClassifier

### Reading the training dataset to Pandas DataFrame

In [24]:
data = pd.read_csv('train.csv')
data.head()

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,Minor_Damage_And_Injuries,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352,7570
1,Minor_Damage_And_Injuries,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350,12128
2,Significant_Damage_And_Fatalities,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364,2181
3,Significant_Damage_And_Serious_Injuries,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728,5946
4,Significant_Damage_And_Fatalities,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883,9054


### Getting the target variables to Y variable

In [25]:
Y = data['Severity']
Y.shape

(10000,)

### Dropoing the irrelevent columns from training data

In [26]:
data = data.drop(columns=['Severity','Accident_ID','Accident_Type_Code'],axis=1)
data.head()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Max_Elevation,Violations,Adverse_Weather_Metric
0,49.223744,14,22,71.285324,0.272118,78.04,31335.476824,3,0.424352
1,62.465753,10,27,72.288058,0.423939,84.54,26024.711057,2,0.352350
2,63.059361,13,16,66.362808,0.322604,78.86,39269.053927,3,0.003364
3,48.082192,11,9,74.703737,0.337029,81.79,42771.499200,1,0.211728
4,26.484018,13,25,47.948952,0.541140,77.16,35509.228515,2,0.176883


### creating the Label Encoder object which will encode the target severities to numerical form

In [33]:
label_encode = LabelEncoder()
y = label_encode.fit_transform(Y)

In [34]:
y.shape

(10000,)

### split the dataset for training and testing purpose

In [35]:
x_train,x_test,y_train,y_test = train_test_split(data,y,test_size = 0.3)

In [213]:
dtc = DecisionTreeClassifier(criterion='entropy',class_weight='balanced',presort=True,max_depth=10)
bag = BaggingClassifier(warm_start=True,n_estimators=150,base_estimator=dtc)
xgbclf = XGBClassifier()
xgbrgclf = XGBRFClassifier()

In [218]:
dtc.fit(x_train,y_train)


DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=True,
                       random_state=None, splitter='best')

In [219]:
predictions = dtc.predict(x_test)
accuracy_score(y_test,predictions)

0.9343333333333333

In [220]:
bag.fit(x_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                        criterion='entropy',
                                                        max_depth=10,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=True,
                                                        random_state=None,
                                                        splitter='best')

In [221]:
predictions = bag.predict(x_test)
accuracy_score(y_test,predictions)

0.9536666666666667

In [125]:
xgbclf.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [106]:
predictions = xgbclf.predict(x_test)
accuracy_score(y_test,predictions)

0.8286666666666667

In [13]:
test_data = pd.read_csv('test.csv')

In [14]:
activity_id = test_data['Accident_ID']
test_data = test_data.drop(columns=['Accident_ID','Accident_Type_Code'])

In [15]:
new_pred = dtc.predict(test_data)

In [16]:
test_categories = label_encode.inverse_transform(new_pred)

In [17]:
result_df = pd.DataFrame({'Accident_ID':activity_id,'Severity':test_categories})

In [18]:
result_df.to_csv('Prediction.csv',index=False)

### Accuracy - 83.58